## Load Libraries

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import sys
sys.path.append("../")
from geopy import distance
import math
import os

### Locations and Distances for government sensors

In [ ]:
location_df = pd.read_csv("/scratch/ab9738/pollution_img/govdata/locations.csv")

location_df = location_df.drop(["source"], axis=1)

columns = [loc for loc in location_df['id']]

distance_df = pd.DataFrame(columns = [columns])

for i, locx in enumerate(columns):
    distance_df.loc[locx] = 0
    for j,locy in enumerate(columns):
        # print(type(location_df.loc[i]['lat']))
        cord_x = (location_df.loc[i]['lat'], location_df.loc[i]['lon'])
        cord_y = (location_df.loc[j]['lat'], location_df.loc[j]['lon'])
        distance_df.at[locx,locy] = distance.distance(cord_x, cord_y).kilometers

distance_df.replace(0, np.nan, inplace=True)

### Finding close-by sensors

In [ ]:
def find_closeby(dist_df, radius):
    location_list = list(dist_df.index)
    num_locs = len(location_list)
    loc_pairs = []
    for i, locx in enumerate(location_list):
        for j in range(i+1, num_locs):
            if(dist_df.loc[locx][location_list[j]]<radius):
                loc_pairs.append((locx,location_list[j]))
    return loc_pairs

In [ ]:
find_closeby(distance_df, 2.0)

### Load the government and kaiterra data

In [ ]:
fulldata = pd.read_csv('/scratch/ab9738/pollution_img/govdata/govdata_1H_20180501_20201101.csv',\
                      index_col='monitor_id', usecols=['monitor_id','timestamp_round','pm10'])

In [ ]:
fulldata

In [ ]:
fulldata = fulldata.loc[~(fulldata==0).all(axis=1)]
fulldata

### Estimate the sensor substitution error

In [ ]:
pairs = find_closeby(distance_df, 2.0)

In [ ]:
pairs

In [ ]:
def mean_rel_abs_error(df1, df2):
    assert(df1.index == df2.index).all()
    if(len(list(df1.index))):
        total_rel_err = 0
        total_valid_len = 0
        for ind in list(df1.index):
            rel_err = abs(df2.loc[ind][0]-df1.loc[ind][0])/df1.loc[ind][0]
            if(not math.isnan(rel_err)):
                total_rel_err += rel_err
                total_valid_len += 1
        mean_err = total_rel_err/total_valid_len
        return mean_err

In [ ]:
def mean_abs_error(df1, df2):
    assert(df1.index == df2.index).all()
    if(len(list(df1.index))):
        total_rel_err = 0
        total_valid_len = 0
        for ind in list(df1.index):
            rel_err = abs(df2.loc[ind][0]-df1.loc[ind][0])
            if(not math.isnan(rel_err)):
                total_rel_err += rel_err
                total_valid_len += 1
        mean_err = total_rel_err/total_valid_len
        return mean_err

In [ ]:
mean_rel_sensubs_err = []
mean_sensubs_err = []

In [ ]:
for indv_pair in pairs:
    df1 = fulldata.loc[indv_pair[0]].set_index('timestamp_round')
    df2 = fulldata.loc[indv_pair[1]].set_index('timestamp_round')
    common_timestamps = list(set(list(df1.index)).intersection(list(df2.index)))
    mean_rel_sensubs_err.append(mean_rel_abs_error(df1.loc[common_timestamps], df2.loc[common_timestamps]))
    mean_rel_sensubs_err.append(mean_rel_abs_error(df2.loc[common_timestamps], df1.loc[common_timestamps]))

In [ ]:
mean_rel_sensubs_err

In [ ]:
for indv_pair in pairs:
    df1 = fulldata.loc[indv_pair[0]].set_index('timestamp_round')
    df2 = fulldata.loc[indv_pair[1]].set_index('timestamp_round')
    common_timestamps = list(set(list(df1.index)).intersection(list(df2.index)))
    mean_sensubs_err.append(mean_abs_error(df1.loc[common_timestamps], df2.loc[common_timestamps]))

In [ ]:
for i in range(len(pairs)):
    print(pairs[i], distance_df.loc[pairs[i][0]][pairs[i][1]], mean_sensubs_err[i])

In [ ]:
distance_df.loc[pairs[0][0]][pairs[0][1]]